In [1]:
import plasmapy

Note that plasmapy can take a very long time to import, about a minute.

In [13]:
import hii_utils
import plasmapy.formulary as pf
import astropy.units as u

## Calculate various parameters for a typical H II region

### Fiducial parameters

As with my other notes, we will normalise to a density of 100 pcc and temperature of 10,000 K. We set the 

In [3]:
L = 10.0 * u.pc
n = 100.0 * u.cm ** -3
T = 1.0e4 * u.K
species = ("p+", "p+")

In [4]:
pf.Knudsen_number(L, T, n, species)

<Quantity 1.79932417e-10>

Mean free path comes out the same for electrons and protons

In [5]:
pf.mean_free_path(T, n, species).cgs

<Quantity 5.55213426e+09 cm>

But it is slightly different for Helium ions

In [6]:
pf.mean_free_path(T, n, ("He++", "p+")).cgs

<Quantity 1.43652977e+09 cm>

And it is a *lot* bigger for high-velocity protons (for example, from stellar wind)

In [7]:
pf.mean_free_path(T, n, species, V=1000 * u.km/u.s).cgs

<Quantity 3.66291304e+16 cm>

By default, the transport coefficients are calculated in the Braginskii model

In [8]:
pf.ion_viscosity(T, n, T, n, "p")

<Quantity [1.07673276e-07, 1.07345943e-07, 1.07345943e-07, 0.00000000e+00,
           0.00000000e+00] Pa s>

But there is the option to use the more accurate Ji and Held (2013) model, although in our case this makes little difference.

In [9]:
pf.ion_viscosity(T, n, T, n, "p", model="Ji-Held")

<Quantity [1.08256591e-07, 1.08256591e-07, 1.08256591e-07, 0.00000000e+00,
           0.00000000e+00] Pa s>

Note that viscosity is a tensor, which may become important once we introduce a magnetic field.

In [10]:
B = 10 * u.microgauss
pf.beta(T, n, B)

<Quantity 34.69949406>

In [11]:
pf.ion_viscosity(T, n, T, n, "p", B=B, model="Ji-Held")

<Quantity [1.08256591e-07, 7.99898369e-20, 3.19958436e-19, 8.64659363e-14,
           1.72931530e-13] Pa s>

In [12]:
pf.ion_viscosity(T, n, T, n, "p", B=B)

<Quantity [1.07673276e-07, 7.99898460e-20, 3.19958580e-19, 8.64659384e-14,
           1.72931547e-13] Pa s>

These are now the tensor viscosity resuts with a more realistic magnetic field